In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("P3").getOrCreate()

25/03/07 15:57:38 WARN Utils: Your hostname, ASDLAPKCH0627 resolves to a loopback address: 127.0.1.1; using 10.10.150.163 instead (on interface wlp0s20f3)
25/03/07 15:57:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/07 15:57:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/07 15:57:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/03/07 15:57:39 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [6]:
df=spark.read.option("header","true").csv("public_response.csv",inferSchema=True)

In [7]:
df.show()

+-------------------+---------------------+----------------------+----------------+-----------------+------------+----------+-----------+
|               film|rotten_tomatoes_score|rotten_tomatoes_counts|metacritic_score|metacritic_counts|cinema_score|imdb_score|imdb_counts|
+-------------------+---------------------+----------------------+----------------+-----------------+------------+----------+-----------+
|          Toy Story|                  100|                    96|              95|               26|           A|       8.3|    1089101|
|       A Bug's Life|                   92|                    91|              78|               23|           A|       7.2|     319596|
|        Toy Story 2|                  100|                   172|              88|               34|          A+|       7.9|     630573|
|     Monsters, Inc.|                   96|                   199|              79|               35|          A+|       8.1|    1000657|
|       Finding Nemo|             

In [8]:
df.printSchema()

root
 |-- film: string (nullable = true)
 |-- rotten_tomatoes_score: integer (nullable = true)
 |-- rotten_tomatoes_counts: integer (nullable = true)
 |-- metacritic_score: integer (nullable = true)
 |-- metacritic_counts: integer (nullable = true)
 |-- cinema_score: string (nullable = true)
 |-- imdb_score: double (nullable = true)
 |-- imdb_counts: integer (nullable = true)



In [9]:
df.columns

['film',
 'rotten_tomatoes_score',
 'rotten_tomatoes_counts',
 'metacritic_score',
 'metacritic_counts',
 'cinema_score',
 'imdb_score',
 'imdb_counts']

In [16]:
from pyspark.ml.feature import VectorAssembler
feature=VectorAssembler(inputCols=["rotten_tomatoes_score","imdb_counts"],outputCol="score_relation")

In [17]:
output=feature.transform(df)

In [18]:
output.show()

+-------------------+---------------------+----------------------+----------------+-----------------+------------+----------+-----------+-----------------+
|               film|rotten_tomatoes_score|rotten_tomatoes_counts|metacritic_score|metacritic_counts|cinema_score|imdb_score|imdb_counts|   score_relation|
+-------------------+---------------------+----------------------+----------------+-----------------+------------+----------+-----------+-----------------+
|          Toy Story|                  100|                    96|              95|               26|           A|       8.3|    1089101|[100.0,1089101.0]|
|       A Bug's Life|                   92|                    91|              78|               23|           A|       7.2|     319596|  [92.0,319596.0]|
|        Toy Story 2|                  100|                   172|              88|               34|          A+|       7.9|     630573| [100.0,630573.0]|
|     Monsters, Inc.|                   96|                   19

In [19]:
final_data=output.select("score_relation","imdb_score")

In [20]:
final_data.show()

+-----------------+----------+
|   score_relation|imdb_score|
+-----------------+----------+
|[100.0,1089101.0]|       8.3|
|  [92.0,319596.0]|       7.2|
| [100.0,630573.0]|       7.9|
| [96.0,1000657.0]|       8.1|
| [99.0,1132877.0]|       8.2|
|  [97.0,828986.0]|       8.0|
|  [75.0,484034.0]|       7.2|
|  [96.0,851229.0]|       8.1|
| [95.0,1226415.0]|       8.4|
| [98.0,1151249.0]|       8.3|
|  [98.0,908222.0]|       8.3|
|  [40.0,225724.0]|       6.2|
|  [79.0,452012.0]|       7.1|
|  [80.0,398903.0]|       7.2|
|  [98.0,834268.0]|       8.1|
|  [75.0,130324.0]|       6.7|
|  [94.0,311793.0]|       7.2|
|  [69.0,128849.0]|       6.7|
|  [97.0,618005.0]|       8.4|
|  [93.0,341353.0]|       7.5|
+-----------------+----------+
only showing top 20 rows



In [21]:
from pyspark.ml.regression import LinearRegression
train_data,test_data=final_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol="score_relation",labelCol="imdb_score")
regressor=regressor.fit(train_data)

25/03/07 17:14:55 WARN Instrumentation: [bcba632e] regParam is zero, which might cause numerical instability and overfitting.
25/03/07 17:14:55 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/03/07 17:14:55 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [22]:
regressor.coefficients

DenseVector([0.028, 0.0])

In [23]:
regressor.intercept

4.602334651229873

In [24]:
pred_result=regressor.evaluate(test_data)

In [25]:
pred_result.predictions.show()

+---------------+----------+------------------+
| score_relation|imdb_score|        prediction|
+---------------+----------+------------------+
|[73.0,140174.0]|       7.0| 6.772081221937828|
|[90.0,168090.0]|       7.6| 7.273112335849904|
|[94.0,311793.0]|       7.2|  7.51435926033068|
|[95.0,158649.0]|       7.0|7.4045962142573645|
+---------------+----------+------------------+

